In [1]:
# Import Dependencies
import pandas as pd
import os

In [2]:
# Create a reference the JSON file desired
json_path = os.path.join('Resources', 'purchase_data.json')

# Read the CSV into a Pandas DataFrame
purchase_data_df = pd.read_json(json_path)

In [3]:
# Print the first five rows of the first data frame to the screen
purchase_data_df.head()

,Age,Gender,Item ID,Item Name,Price,SN
0,38,Male,165,Bone Crushing Silver Skewer,3.37,Aelalis34
1,21,Male,119,"Stormbringer, Dark Blade of Ending Misery",2.32,Eolo46
2,34,Male,174,Primitive Blade,2.46,Assastnya25
3,21,Male,92,Final Critic,1.36,Pheusrical25
4,23,Male,63,Stormfury Mace,1.27,Aela59


In [4]:
#Counting the total number of unique players
purchase_data_df["SN"].nunique()

573

In [5]:
#Creating the values for the purchasing analysis summary table
unique_items = purchase_data_df["Item Name"].nunique()
avg_pur_price = purchase_data_df["Price"].mean()
total_purcahse = purchase_data_df["Price"].count()
total_rev = purchase_data_df["Price"].sum()

#Create summary table for purchasing analysis
summary_df = pd.DataFrame ({"Unique Items": [unique_items], 
                           "Average Purchase Price": [avg_pur_price], 
                           "Number of Purchases": [total_purcahse], 
                           "Total Revenue": [total_rev]})

#Arrange columns in desired order and print summary table
summary_df_2 = summary_df[["Unique Items", "Average Purchase Price", "Number of Purchases", "Total Revenue"]]
summary_df_2

summary_df_2.style.format({"Average Purchase Price": "${:.2f}", "Total Revenue": "${:.2f}"})

,Unique Items,Average Purchase Price,Number of Purchases,Total Revenue
0,179,$2.93,780,$2286.33


In [6]:
#Gender Analytics
total_gender = purchase_data_df["Gender"].count()
male = purchase_data_df["Gender"].value_counts()['Male']
female = purchase_data_df["Gender"].value_counts()['Female']
other = purchase_data_df["Gender"].value_counts()['Other / Non-Disclosed']
male_percent = (male/total_gender) * 100
female_percent = (female/total_gender) * 100
other_percent = (other/total_gender) * 100

#Create gender analytics summary table
summary_gender_df = pd.DataFrame ({"Percentage of Players": [male_percent, female_percent, other_percent], 
                                  "Total Count": [male, female, other]}, index = ["Male", "Female", "Non-Specific"])

summary_gender_df.style.format({"Percentage of Players": "%{:.2f}"})

summary_gender_df

,Percentage of Players,Total Count
Male,81.153846,633
Female,17.435897,136
Non-Specific,1.410256,11


In [7]:
#Purchasing Analysis by Gender

pur_gender = purchase_data_df.groupby(["Gender"])

pur_count_gender = pur_gender["SN"].count()

avg_pur_price = pur_gender["Price"].mean()

tot_pur_value = pur_gender["Price"].sum()

duplicate_data = purchase_data_df.drop_duplicates(subset='SN', keep="first")
grouped_duplicate_data = duplicate_data.groupby(["Gender"])

norm_total = (pur_gender["Price"].sum() / grouped_duplicate_data["SN"].count())

summary_analysis_gender_df = pd.DataFrame({ "Purchase Count":pur_count_gender, 
                                            "Average Purchase Price": avg_pur_price, 
                                            "Total Purchase Value": tot_pur_value, 
                                            "Normalized Total": norm_total })

summary_analysis_gender_df

summary_analysis_gender_df.style.format({"Total Purchase Value": '${:.2f}', "Average Purchase Price": '${:.2f}', "Normalized Total": '${:.2f}'})    


,Average Purchase Price,Normalized Total,Purchase Count,Total Purchase Value
Gender,,,,
Female,$2.82,$3.83,136,$382.91
Male,$2.95,$4.02,633,$1867.68
Other / Non-Disclosed,$3.25,$4.47,11,$35.74


In [8]:
#Age Demographics by Binning

bins = [0,10,15,20,25,30,35,40,200]
bins_list = ["< 10", "10-14", "15-19", "20-24", "25-29", "30-34", "35-39", "40+"]

#Bin dataframe
bin_df = purchase_data_df.copy()
bin_df["Age Groups"] = pd.cut(bin_df["Age"], bins, labels =bins_list)
bin_cut = pd.cut(bin_df["Age"], bins, labels = bins_list)
grouped_bin_df = bin_df.groupby(["Age Groups"])

pur_count_age = grouped_bin_df["Age"].count()
avg_price_age = grouped_bin_df["Price"].mean()
tot_pur_age = grouped_bin_df["Price"].sum() 

duplicate = purchase_data_df.drop_duplicates(subset = 'SN', keep = "first")
duplicate["Age Groups"] = pd.cut(duplicate["Age"], bins, labels = bins_list)
duplicate = duplicate.groupby(["Age Groups"])

norm_total_bin_df = (grouped_bin_df["Price"].sum()/duplicate["SN"].count())
norm_total_bin_df

age_bin_df = pd.DataFrame({ "Purchase Count": pur_count_age,
                            "Average Purchase Price": avg_price_age,
                            "Total Purchase Value": tot_pur_age,
                            "Normalized Total": norm_total_bin_df})

age_bin_df

age_bin_df.style.format({"Total Purchase Value": '${:.2f}', "Average Purchase Price": '${:.2f}', "Normalized Total": '${:.2f}'})    


/anaconda3/envs/PythonData/lib/python3.6/site-packages/ipykernel_launcher.py:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


,Average Purchase Price,Normalized Total,Purchase Count,Total Purchase Value
Age Groups,,,,
< 10,$3.02,$4.39,32,$96.62
10-14,$2.87,$4.15,78,$224.15
15-19,$2.87,$3.80,184,$528.74
20-24,$2.96,$3.86,305,$902.61
25-29,$2.89,$4.23,76,$219.82
30-34,$3.07,$4.05,58,$178.26
35-39,$2.90,$5.10,44,$127.49
40+,$2.88,$2.88,3,$8.64


In [9]:
#Top Spenders

SN_group = purchase_data_df.groupby(["SN"])

SN_group_count = SN_group["Item ID"].count()
SN_group_total = SN_group["Price"].sum()
SN_group_Average = (SN_group_total/SN_group_count)

spender_df = pd.DataFrame({"Purchase Count": SN_group_count, 
                      "Average Purchase Price": SN_group_Average,
                      "Total Purchase Value":SN_group_total})

spender_df = spender_df.sort_values("Total Purchase Value", ascending = False)

spender_df.style.format({"Total Purchase Value": '${:.2f}', "Average Purchase Price": '${:.2f}'})

spender_df.head(5)

,Average Purchase Price,Purchase Count,Total Purchase Value
SN,,,
Undirrala66,3.412000,5,17.06
Saedue76,3.390000,4,13.56
Mindimnya67,3.185000,4,12.74
Haellysu29,4.243333,3,12.73
Eoda93,3.860000,3,11.58


In [10]:
#Most Popular Items

top5_items_ID = pd.DataFrame(purchase_data_df.groupby("Item ID")["Item ID"].count())

top5_items_ID.sort_values("Item ID", ascending = False, inplace = True)

top5_items_ID = top5_items_ID.iloc[0:5][:]

top5_items_total = pd.DataFrame(purchase_data_df.groupby("Item ID")["Price"].sum())

top5_items = pd.merge(top5_items_ID, top5_items_total, left_index = True, right_index = True)

no_dup_items = purchase_data_df.drop_duplicates(["Item ID"], keep = 'last')

top5_merge_ID = pd.merge(top5_items, no_dup_items, left_index = True, right_on = "Item ID")

top5_merge_ID = top5_merge_ID[["Item ID", "Item Name", "Item ID_x", "Price_y", "Price_x"]]

top5_merge_ID.set_index(["Item ID"], inplace = True)

top5_merge_ID.rename(columns =  {"Item ID_x": "Purchase Count", 
                                 "Price_y": "Item Price", 
                                 "Price_x": "Total Purchase Value"}, inplace=True)

top5_merge_ID.style.format({"Item Price": '${:.2f}', "Total Purchase Value": '${:.2f}'})

,Item Name,Purchase Count,Item Price,Total Purchase Value
Item ID,,,,
39,"Betrayal, Whisper of Grieving Widows",11,$2.35,$25.85
84,Arcane Gem,11,$2.23,$24.53
31,Trickster,9,$2.07,$18.63
175,Woeful Adamantite Claymore,9,$1.24,$11.16
13,Serenity,9,$1.49,$13.41


In [11]:
#Most Profitable

top5_profit = pd.DataFrame(purchase_data_df.groupby("Item ID")["Price"].sum())
top5_profit.sort_values("Price", ascending = False, inplace = True)

top5_profit = top5_profit.iloc[0:5][:]

pur_count_profit = pd.DataFrame(purchase_data_df.groupby("Item ID")["Item ID"].count())

top5_profit = pd.merge(top5_profit, pur_count_profit, left_index = True, right_index = True, how = 'left')
top5_merge_profit = pd.merge(top5_profit, no_dup_items, left_index = True, right_on = "Item ID", how = 'left')
top5_merge_profit = top5_merge_profit[["Item ID", "Item Name", "Item ID_x", "Price_y","Price_x"]]
top5_merge_profit.set_index(["Item ID"], inplace=True)
top5_merge_profit.rename(columns = {"Item ID_x": "Purchase Count", 
                                    "Price_y": "Item Price", 
                                    "Price_x": "Total Purchase Value"}, inplace = True)
top5_merge_profit.style.format({"Item Price": '${:.2f}', "Total Purchase Value": '${:.2f}'})

,Item Name,Purchase Count,Item Price,Total Purchase Value
Item ID,,,,
34,Retribution Axe,9,$4.14,$37.26
115,Spectral Diamond Doomblade,7,$4.25,$29.75
32,Orenmir,6,$4.95,$29.70
103,Singed Scalpel,6,$4.87,$29.22
107,"Splitter, Foe Of Subtlety",8,$3.61,$28.88
